In [1]:
print("Working")

Working


In [15]:
import gc
import torch
import weakref

def cleanup_gpu_memory(obj=None, verbose: bool = False):

    if not torch.cuda.is_available():
        if verbose:
            print("[INFO] CUDA is not available. No GPU cleanup needed.")
        return

    def get_memory_stats():
        allocated = torch.cuda.memory_allocated()
        reserved = torch.cuda.memory_reserved()
        return allocated, reserved

    torch.cuda.synchronize()

    if verbose:
        alloc, reserv = get_memory_stats()
        print(f"[Before] Allocated: {alloc / 1024**2:.2f} MB | Reserved: {reserv / 1024**2:.2f} MB")

    # Ensure we drop all strong references
    if obj is not None:
        ref = weakref.ref(obj)
        del obj
        if ref() is not None and verbose:
            print("[WARNING] Object not fully garbage collected yet.")

    gc.collect()
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()

    torch.cuda.synchronize()

    if verbose:
        alloc, reserv = get_memory_stats()
        print(f"[After]  Allocated: {alloc / 1024**2:.2f} MB | Reserved: {reserv / 1024**2:.2f} MB")

cleanup_gpu_memory(None, verbose=True)

[Before] Allocated: 474.09 MB | Reserved: 814.00 MB
[After]  Allocated: 474.09 MB | Reserved: 738.00 MB


In [18]:
import os
from roboflow import Roboflow
rf = Roboflow(api_key=os.environ["ROBOFLOW_API_KEY"])
project = rf.workspace("workspace-5ujvu").project("basketball-players-fy4c2-vfsuv")
version = project.version(17)
dataset = version.download("yolov12")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Basketball-Players-17 in yolov12:: 100%|██████████| 652/652 [00:00<00:00, 10530.01it/s]


In [19]:
import os
from pathlib import Path

BASE = Path("Basketball-Players-17")
SETS = ["train", "valid", "test"]

PLAYER_ID = "4"
def clean_and_convert_labels(label_file):
    with open(label_file, "r") as f:
        lines = f.readlines()

    valid_rows = []
    for ln in lines:
        stripped = ln.strip()
        if stripped.startswith(PLAYER_ID):
            parts = stripped.split()
            parts[0] = "0"
            valid_rows.append(" ".join(parts) + "\n")

    if not valid_rows:
        os.remove(label_file)
        print(f"Deleted empty label file: {label_file}")
        return

    with open(label_file, "w") as f:
        f.writelines(valid_rows)


def sync_images_and_labels(images_folder, labels_folder):
    """Ensure each image has a matching label, and vice versa."""
    image_files = {p.stem for p in images_folder.glob("*") if p.suffix.lower() in [".jpg", ".png", ".jpeg"]}
    label_files = {p.stem for p in labels_folder.glob("*.txt")}

    for name in label_files - image_files:
        file = labels_folder / f"{name}.txt"
        os.remove(file)
        print(f"Deleted label without matching image: {file}")

    for name in image_files - label_files:
        for ext in [".jpg", ".png", ".jpeg"]:
            file = images_folder / f"{name}{ext}"
            if file.exists():
                os.remove(file)
                print(f"Deleted image without matching label: {file}")


for split in SETS:
    print(f"\n--- Processing {split} ---")
    images = BASE / split / "images"
    labels = BASE / split / "labels"

    for txt in labels.glob("*.txt"):
        clean_and_convert_labels(txt)

    sync_images_and_labels(images, labels)


--- Processing train ---
Deleted empty label file: Basketball-Players-17/train/labels/youtube-86_jpg.rf.2eef19a5a7ec14e9f558b795dca6f55d.txt
Deleted empty label file: Basketball-Players-17/train/labels/youtube-108_jpg.rf.be8562178799253b0ee2b6bb33732c0e.txt
Deleted empty label file: Basketball-Players-17/train/labels/youtube-233_jpg.rf.48e360584d4a0b5bc3124893a46a8056.txt
Deleted image without matching label: Basketball-Players-17/train/images/youtube-86_jpg.rf.2eef19a5a7ec14e9f558b795dca6f55d.jpg
Deleted image without matching label: Basketball-Players-17/train/images/youtube-108_jpg.rf.be8562178799253b0ee2b6bb33732c0e.jpg
Deleted image without matching label: Basketball-Players-17/train/images/youtube-233_jpg.rf.48e360584d4a0b5bc3124893a46a8056.jpg

--- Processing valid ---

--- Processing test ---


In [20]:
import cv2
import os

# --- INPUT PATHS ---
image_path = "Basketball-Players-17/train/images/frame_9024_jpg.rf.74a973ec4e2d5c654af9d6d871d2f4e9.jpg"
label_path = image_path.replace("images", "labels").replace(".jpg", ".txt")

# --- OUTPUT PATH ---
output_path = image_path.replace(".jpg", "_annotated.jpg")

# Load image
img = cv2.imread(image_path)
h, w, _ = img.shape

# Read label file
if os.path.exists(label_path):
    with open(label_path, "r") as f:
        labels = f.readlines()
else:
    raise FileNotFoundError(f"No label file found for: {image_path}")

# Draw each bounding box
for line in labels:
    cls, x, y, bw, bh = line.strip().split()
    x, y, bw, bh = map(float, [x, y, bw, bh])

    # Convert YOLO format → pixel coordinates
    x1 = int((x - bw/2) * w)
    y1 = int((y - bh/2) * h)
    x2 = int((x + bw/2) * w)
    y2 = int((y + bh/2) * h)

    # Draw box + label
    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
    cv2.putText(img, cls, (x1, y1 - 5),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

# Save annotated image
cv2.imwrite(output_path, img)

print("Saved annotated image to:", output_path)


Saved annotated image to: Basketball-Players-17/train/images/frame_9024_jpg.rf.74a973ec4e2d5c654af9d6d871d2f4e9_annotated.jpg


In [3]:
import os
import wandb
from dotenv import load_dotenv
load_dotenv()

WNB_API_KEY = os.getenv("WNB_API_TOKEN")
if WNB_API_KEY is None:
    raise EnvironmentError(f"Environment variable {WNB_API_KEY} is not set")
wandb.login(key=WNB_API_KEY)

!wandb status

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/maso/.netrc
wandb: Currently logged in as: mosmar99 to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Current Settings
{
  "_extra_http_headers": null,
  "_proxies": null,
  "api_key": null,
  "base_url": "https://api.wandb.ai",
  "entity": null,
  "git_remote": "origin",
  "ignore_globs": [],
  "organization": null,
  "project": null,
  "root_dir": null,
  "section": "default"
}


In [15]:
import wandb

run = wandb.init(
    project="basketball_tracker",
    entity="baas_group5",
    job_type=f"player_dataset"
)

artifact = wandb.Artifact(
    name="player_dataset",
    type="dataset",
    description="Player dataset upload"
)

artifact.add_dir("Basketball-Players-17")

wandb.log_artifact(artifact)

wandb.finish()


wandb: Adding directory to artifact (Basketball-Players-17)... Done. 1.0s


In [ ]:
import wandb
run = wandb.init(
    project="basketball_tracker",
    entity="baas_group5",
    job_type=f"player_dataset"
)
artifact = run.use_artifact('baas_group5/basketball_tracker/player_dataset:v3', type='dataset')
artifact_dir = artifact.download()

wandb:   637 of 637 files downloaded.  


In [ ]:
import wandb
from ultralytics import YOLO

# -------------------------------
# Config
# -------------------------------
MODEL_NAME = "yolo12l"     # Larger model = higher accuracy
IMG_SIZE = 640             # Better for small players
EPOCHS = 150               # Your curves indicate more training helps

# -------------------------------
# Init W&B Run
# -------------------------------
run = wandb.init(
    project="basketball_tracker",
    entity="baas_group5",
    job_type=f"player_finetune_{MODEL_NAME}"
)

# -------------------------------
# Load dataset artifact (with lineage)
# -------------------------------
dataset_artifact = run.use_artifact(
    "baas_group5/basketball_tracker/player_dataset:v3",
    type="dataset"
)
artifact_dir = dataset_artifact.download()

# -------------------------------
# Load model
# -------------------------------
model = YOLO(f"PATH_TO_WEIGHTS/{MODEL_NAME}.pt")


# -------------------------------
# W&B Metric Callbacks
# -------------------------------
def extract_metrics(metrics):
    if metrics is None:
        return {}
    if isinstance(metrics, dict):
        return metrics
    if hasattr(metrics, "results_dict") and isinstance(metrics.results_dict, dict):
        return metrics.results_dict
    if hasattr(metrics, "results_dict") and callable(metrics.results_dict):
        return metrics.results_dict()
    return {}

def on_train_epoch_end(trainer):
    m = extract_metrics(trainer.metrics)
    if m:
        wandb.log(m)

def on_val_end(trainer):
    m = extract_metrics(trainer.metrics)
    if m:
        wandb.log({f"val/{k}": v for k, v in m.items()})

model.add_callback("on_train_epoch_end", on_train_epoch_end)
model.add_callback("on_val_end", on_val_end)


# -------------------------------
# Train with optimized params
# -------------------------------
results = model.train(
    data=artifact_dir + "/data.yaml",
    epochs=EPOCHS,
    imgsz=IMG_SIZE,
    lr0=0.01,
    lrf=0.001,
    cos_lr=True,
    dropout=0.05,

    mosaic=1.0,
    mixup=0.2,
    degrees=5.0,
    translate=0.1,
    scale=0.5,
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
)

# -------------------------------
# Log model artifact
# -------------------------------
best_path = f"{results.save_dir}/weights/best.pt"

model_artifact = wandb.Artifact(
    f"{MODEL_NAME}-imgsz{IMG_SIZE}-player-finetuned",
    type="model"
)
model_artifact.add_file(best_path)

run.log_artifact(model_artifact)
run.finish()


wandb:   637 of 637 files downloaded.  


New https://pypi.org/project/ultralytics/8.3.235 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.223 🚀 Python-3.12.12 torch-2.9.1+cu130 CUDA:0 (NVIDIA GeForce RTX 5090, 32607MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/home/maso/learn/robo_bt_tutorial/artifacts/player_dataset:v3/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=150, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=PATH_TO_WEIGHTS/yolo12l.pt, momentum=0.937

metrics/mAP50(B),▁▁▁▁▆▅▆▇▇▇██▇███████████████████████████
metrics/mAP50-95(B),▁▁▁▂▃▅▅▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇██████████████████
metrics/precision(B),▁▁▆▇▇▇▇▇▇▇█▇████▇███████████████████████
metrics/recall(B),▁▁▂▅▅▆▇▇▇▇▇▇▇▇▇█▇▇██████████████████████
val/box_loss,█▅▆▅▅▅▄▄▃▃▃▂▃▂▂▂▂▂▂▂▂▂▁▂▂▁▁▁▁▁▁▁▁▁▁
val/cls_loss,█▄▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val/dfl_loss,▁█▆▅▅▅▅▅▅▅▅▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄
val/fitness,▁▁▁▂▂▃▄▅▅▅▆▅▇▇▇▇▇▇▇▇▇███████████████████
val/metrics/mAP50(B),▁▁▁▃▄▆▇▇▇▇▇██▇██████████████████████████
val/metrics/mAP50-95(B),▁▁▄▃▄▆▆▆▇▆▇▇▇▇▇▇▇▇█▇▇███████████████████
+2,...


In [16]:
import cv2
from ultralytics import YOLO

# ---------------------------------------
# Load trained model
# ---------------------------------------
model = YOLO("runs/detect/train14/weights/best.pt")

# Input/output video paths
input_path = "videos/video_1.mp4"
output_path = "videos/video_1_bbox.mp4"

# ---------------------------------------
# Open input video
# ---------------------------------------
cap = cv2.VideoCapture(input_path)

if not cap.isOpened():
    raise IOError(f"Cannot open video: {input_path}")

# Get video properties
fps = cap.get(cv2.CAP_PROP_FPS)
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# ---------------------------------------
# Create output video writer
# ---------------------------------------
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
writer = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

# ---------------------------------------
# Process frame-by-frame
# ---------------------------------------
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLO inference
    results = model(frame, conf=0.3, iou=0.6)  # tune if needed

    # Draw boxes
    annotated_frame = results[0].plot()   # draws bounding boxes, labels, confidences

    # Write to output video
    writer.write(annotated_frame)

# ---------------------------------------
# Cleanup
# ---------------------------------------
cap.release()
writer.release()

print(f"Saved video with bounding boxes → {output_path}")



0: 384x640 7 Players, 57.0ms
Speed: 1.6ms preprocess, 57.0ms inference, 3.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 Players, 22.8ms
Speed: 1.0ms preprocess, 22.8ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 Players, 43.3ms
Speed: 1.4ms preprocess, 43.3ms inference, 3.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 Players, 10.1ms
Speed: 0.8ms preprocess, 10.1ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 Players, 53.7ms
Speed: 0.9ms preprocess, 53.7ms inference, 2.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 Players, 12.2ms
Speed: 0.7ms preprocess, 12.2ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 Players, 10.2ms
Speed: 1.0ms preprocess, 10.2ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 Players, 52.4ms
Speed: 0.9ms preprocess, 52.4ms inference, 3.1ms postprocess per image at shape (